<a href="https://colab.research.google.com/github/namrmino/lguplus_2021/blob/main/sykim/%EC%A4%80%EA%B3%B5_%EA%B4%80%EB%A0%A8_%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%89%B4%EC%8A%A4_%ED%81%AC%EB%A1%A4%EB%A7%81_2021_03_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from pandas import DataFrame
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import numpy as np

## 1. 네이버 뉴스 기사 제목 크롤링 ( 검색 키워드 "준공", 기간 "1일" )
### 01. 초기 URL 설정 및 

In [ ]:
# 네이버 뉴스 URL 저장
# 검색 키워드 "준공", 제외 키워드"준공식", 기간 "1일" 
news_url = 'https://search.naver.com/search.naver?where=news&query=%EC%A4%80%EA%B3%B5%20-%EC%A4%80%EA%B3%B5%EC%8B%9D&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=4&ds=&de=&docid=&nso=so%3Ar%2Cp%3A1d%2Ca%3Aall&mynews=0&refresh_start=0&related=0'

req = requests.get(news_url, verify = False)
soup = BeautifulSoup(req.text, 'html.parser')

C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


### 02. 변수 초기화
* news_dict : 뉴스 제목 및 해당 뉴스의 url을 저장할 딕셔너리
* idx : 크롤링한 뉴스 기사 인덱스
* cur_page : 크롤링한 뉴스 페이지

In [ ]:
news_dict = {} 
idx = 0 
cur_page = 1

### 03. 기사 제목 및 본문 내용(요약) 첫번째 페이지 부터 마지막 페이지까지 크롤링

In [ ]:
print()
print('크롤링 중...')
  
while True:
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
#     a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
  
    for area in area_list:
        # 크롤링한 날짜 및 시간 저장
        date = str(datetime.now()) 
        date = date[:date.rfind(':')]
        
        # 기사 제목 및 url이 저장되어 있는 부분
        n = area.find('a', {'class' : 'news_tit'})
        # 본문 내용(요약)이 저장되어 있는 부분
        t = area.find('a', {'class' : 'api_txt_lines dsc_txt_wrap'})
        
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href'),
                          'text' : t.get_text(),
                          'date' : str(date)}
        idx += 1
    
    cur_page += 1
    
    # 페이지 url 저장되어 있는 부분
    pages = soup.find('div', {'class' : 'sc_page_inner'})
    
    # 마지막 페이지까지 크롤링 하기 위한 코드
    stop_num = 0
    break_var = False
    
    for p in pages.find_all('a'):
        if p.text == str(cur_page):
            next_page_url = p.get('href')
            break
        else:
            stop_num += 1
            if stop_num == len(pages.find_all('a')):
                print('크롤링 끝')
                break_var = True
                break
    if break_var == True:
        break
    
#     반복문 제어(크롤링 할 페이지를 정해줌)
#     if cur_page == 15:
#         break
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url, verify = False)
    soup = BeautifulSoup(req.text, 'html.parser')


크롤링 중...


C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionp

C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


크롤링 끝


In [ ]:
# 딕셔너리 데이터 프레임 변환
news_df = DataFrame(news_dict).T

## 2. 뉴스 기사에서 지역명(시군구, 읍면동)과 날짜(년, 월, 일) 추출
### 01. 행정구역 리스트 생성
 - 행정구역 형식이 추가되어야 할 수도 있음

In [ ]:
##########
# (1) 국가에서 제공하는 행정구역 정보 입력
adm_dist = pd.read_csv('sigungu.txt', sep ='\t')

adm_dist2 = adm_dist.assign(Test=lambda x: x['시군구명칭'] + ' ' + x['시군구'].fillna(''))[['시도명칭', 'Test', '읍면동명칭']]
adm_dist2['Test'] = adm_dist2['Test'].map(lambda x: x.rstrip())
adm_dist2
##########
# (2) 기존의 행정구역 명칭을 뉴스에서 주로 사용되는 형식으로 변환하는 함수 생성
# 시, 군, 구 제거 함수( 뉴스에서 시군구를 제외하고 작성하는 경우가 다수 존재)
def f1(x):
    if x in ['강화군', '고령군', '동작구', '사상구', '수영구', '동안구',
            '영광군', '예산군', '상당구', '달성군']:
        return x
    elif len(x) > 2:
        return x[:-1]
    else :
        return x

# "--가 --동" 형식으로 되어있는 읍면동 명칭 정리 함수 (주로 --동으로 작성됨 )
# 예외적인 명칭에 대해서는 emd2 리스트로 정리
f2 = lambda x : re.sub('[0-9]{1,5}가', '',
                       re.sub('[0-9]{1,4}동', '동',
                              re.sub('·', '', x)))
##########
# (3) 각 행정구역 명칭 뉴스 형식에 맞게 전처리 및 replace dictionary 생성
## sido
sido_origin = list(set(adm_dist['시도명칭']))

sido_add = [
            '서울시', '서울', '세종시', '세종',
            '경북', '경남', '충북', '충남', '전북', '전남',
            '경상도', '충청도', '전라도',
            '울산시', '울산', '대전시', '대전', '대구시', '대구',
            '부산시', '부산', '인천시', '인천', '광주시', '광주',
            '제주도', '제주', '경기', '강원'
            ]
sido = sido_origin + sido_add

sido_dic = {# sido_origin : sido_add
            '서울시' : '서울특별시',
            '서울' : '서울특별시',
            '세종시' : '세종특별자치시',
            '세종' : '세종특별자치시',
            '경북' : '경상북도',
            '경남' : '경상남도',
            '충북' : '충청북도',
            '충남' : '충청남도',
            '전북' : '전라북도',
            '전남' : '전라남도',
            '강원' : '강원도',
            '경기' : '경기도',
            '울산시' : '울산광역시',
            '울산' : '울산광역시',
            '대구시' : '대구광역시',
            '대구' : '대구광역시',
            '광주시' : '광주광역시',
            '광주' : '광주광역시',
            '대전시' : '대전광역시',
            '대전' : '대전광역시',
            '인천시' : '인천광역시',
            '인천' : '인천광역시',
            '부산시' : '부산광역시',
            '부산' : '부산광역시',
            '제주도' : '제주특별자치도',
            '제주' : '제주특별자치도',
           }

## sgg
sgg_origin = list(set(adm_dist['시군구명칭']))
sgg2_origin = list(set(adm_dist['시군구']))[1:] # 결측값 제거

sgg_add = list(map(f1, sgg_origin))
sgg2_add = list(map(f1, sgg2_origin))

sgg = sgg_origin + sgg_add + sgg2_origin + sgg2_add 

sgg_dic = { replace:origin for replace, origin in zip(sgg_add + sgg2_add, sgg_origin + sgg2_origin) }

## emd
emd_origin = list(set(adm_dist['읍면동명칭']))
emd_add = list(map(f2, emd_origin))
# emd_2 = list(map(f1, emd))

emd_add2 = ['종로1가', '종로2가', '종로3가', '종로4가', '종로5가', '종로6가',
            '이태원', '호암동', '칠금동', '목행동', '성내동', '송도', '을지로']

emd = emd_origin + emd_add + emd_add2

emd_dic1 = { replace:origin for replace, origin in zip(emd_add, emd_origin) }
emd_dic2 = {# sido_origin : sido_add
            '종로1가' : '종로1·2·3·4가동',
            '종로2가' : '종로1·2·3·4가동',
            '종로3가' : '종로1·2·3·4가동',
            '종로4가' : '종로1·2·3·4가동',
            '종로5가' : '종로5·6가동',
            '종로6가' : '종로5·6가동',
            '이태원' : '이태원2동',
            '호암동' : '호암·직동',
            '칠금동' : '칠금·금릉동',
            '목행동' : '목행·용탄동',
            '성내동' : '성내·충인동',
            '을지로' : '을지로동'        
            }
emd_dic = emd_dic1.update(emd_dic2)

#########
# (4) 준공 지역 키워드 리스트 생성
adm_list = list(set(sido + sgg + emd))
adm_list.sort(key=len)
adm_list = adm_list[::-1]

In [ ]:
sgg_two = adm_dist[adm_dist['시군구'].notnull()][['시군구명칭', '시군구']].drop_duplicates()
sgg_two['시군구명칭2'] = list(sgg_two['시군구명칭'].map(f1))
sgg_two['시군구2'] = list(sgg_two['시군구'].map(f1))

sgg_pair = []

for i in [0, 2]:
    for j in range(0, len(sgg_two)):
        list_temp = [sgg_two.iloc[j, i], sgg_two.iloc[j, 1]]
        sgg_pair.append(list_temp)
        list_temp = [sgg_two.iloc[j, i], sgg_two.iloc[j, 3]]
        sgg_pair.append(list_temp)

sgg_pair

[['수원시', '장안구'],
 ['수원시', '장안'],
 ['수원시', '권선구'],
 ['수원시', '권선'],
 ['수원시', '팔달구'],
 ['수원시', '팔달'],
 ['수원시', '영통구'],
 ['수원시', '영통'],
 ['성남시', '수정구'],
 ['성남시', '수정'],
 ['성남시', '중원구'],
 ['성남시', '중원'],
 ['성남시', '분당구'],
 ['성남시', '분당'],
 ['안양시', '만안구'],
 ['안양시', '만안'],
 ['안양시', '동안구'],
 ['안양시', '동안구'],
 ['안산시', '상록구'],
 ['안산시', '상록'],
 ['안산시', '단원구'],
 ['안산시', '단원'],
 ['고양시', '덕양구'],
 ['고양시', '덕양'],
 ['고양시', '일산동구'],
 ['고양시', '일산동'],
 ['고양시', '일산서구'],
 ['고양시', '일산서'],
 ['용인시', '처인구'],
 ['용인시', '처인'],
 ['용인시', '기흥구'],
 ['용인시', '기흥'],
 ['용인시', '수지구'],
 ['용인시', '수지'],
 ['청주시', '상당구'],
 ['청주시', '상당구'],
 ['청주시', '서원구'],
 ['청주시', '서원'],
 ['청주시', '흥덕구'],
 ['청주시', '흥덕'],
 ['청주시', '청원구'],
 ['청주시', '청원'],
 ['천안시', '동남구'],
 ['천안시', '동남'],
 ['천안시', '서북구'],
 ['천안시', '서북'],
 ['전주시', '완산구'],
 ['전주시', '완산'],
 ['전주시', '덕진구'],
 ['전주시', '덕진'],
 ['포항시', '남구'],
 ['포항시', '남구'],
 ['포항시', '북구'],
 ['포항시', '북구'],
 ['창원시', '의창구'],
 ['창원시', '의창'],
 ['창원시', '성산구'],
 ['창원시', '성산'],
 ['창원시', '마산합포구'],
 ['창원시', '마산합포'],
 [

In [ ]:
# 리스트(data) 안에 target이 있는지 확인하는 함수 생성

# def find_index(data, target):
#     res = []
#     lis = data
#     while True:
#         try:
#             res.append(lis.index(target) + (res[-1]+1 if len(res)!=0 else 0)) #+1의 이유 : 0부터 시작이니까
#             lis = data[res[-1]+1:]
#         except:
#             break     
#     return res

# print(find_index(adm_list, '예산'))

In [ ]:
reg_list = []

# 리스트로 뽑아서 news_df에 넣는 것이 아닌 바로 넣을 수 있으면 좋겠다
for t in news_df['title']: 
    reg_list.append(re.findall(r"(?=(" + '|'.join(adm_list) + r"))", t))

news_df['region_keyword'] = reg_list

pd.set_option('display.max_row', None)
news_df.loc[:,['region_keyword']]
# news_df['region_keyword']

,region_keyword
0,[광양]
1,[김천시]
2,[진주시]
3,[괴산군]
4,"[서산시, 부석면]"
5,[구례군]
6,[세종]
7,"[춘천, 춘천]"
8,[춘천]
9,"[완주군, 소양면]"


In [ ]:
news_df[news_df['region_keyword'] == '[]']

,title,url,text,date,region_keyword


In [ ]:
reg_list2 = []

# 리스트로 뽑아서 news_df에 넣는 것이 아닌 바로 넣을 수 있으면 좋겠다
for t in news_df['text']: 
    reg_list2.append(re.findall(r"(?=(" + '|'.join(adm_list) + r"))", t))

news_df['region_keyword2'] = reg_list2

# pd.set_option('display.max_row', None)
news_df.loc[:,['text', 'region_keyword2']]

news_df.loc[:,['title', 'text', 'region_keyword', 'region_keyword2']].to_excel('text_test2.xlsx')
# news_df['region_keyword2']

In [ ]:
# (성동구, 동구)와 같이 하나의 단어에서 두 개가 뽑히는 것 정리
for i in range(0, len(news_df['region_keyword'])):
    if len(news_df['region_keyword'][i]) >= 2:
        for j in range(0, len(news_df['region_keyword'][i])-1):
            if news_df['region_keyword'][i][j][1:] == news_df['region_keyword'][i][j+1]:
                del news_df['region_keyword'][i][j+1]
                break

In [ ]:
news_df['region_total'] = news_df['region_keyword'] +  news_df['region_keyword2']

In [ ]:
sido_list = []
sgg_list = []
emd_list = []

sido_p = re.compile('.*(시|도)$')
sgg_p = re.compile('.*(시|군|구)$')

for reg_list in news_df['region_total']:
    sido_temp = []
    sgg_temp = []
    emd_temp = []
    
    sido_cnt = 0
    sgg_cnt = 0
    
    sido_p = re.compile('.*(시|도)$')
    sgg_p = re.compile('.*(시|군|구)$')
    
    if len(reg_list) > 0:
        for reg_word in reg_list:
            if reg_word in sido:
                sido_temp.append(reg_word)
                if sido_p.search(reg_word) != None:
                    sido_cnt += 1
                    
            elif reg_word in sgg:
                sgg_temp.append(reg_word)
                if sgg_p.search(reg_word) != None:
                    sgg_cnt += 1
                
            elif reg_word in emd:
                emd_temp.append(reg_word)
                
    if len(sido_temp) > 1 & sido_cnt > 0:
#         print(sido_temp)
#         print(sido_cnt)
        temp_list = []
        for word in sido_temp:
            temp = sido_p.search(word)
            if temp != None:
                temp_list.append(temp.group())
        sido_temp = temp_list
    
#     if len(sgg_temp) > 1:
#         for sgg_p_word in sgg_pair:
#             a = list(set(sgg_temp) & set(sgg_p_word))
#             if len(a) == 2:
#                 sgg_temp = list(' '.join(list(a)))
#                 break
                
    if len(sgg_temp) > 1 & sgg_cnt > 0:
        print(sgg_temp)
        print(sgg_cnt)
        temp_list = []
        for word in sgg_temp:
            temp = sgg_p.search(word)
            if temp != None:
                temp_list.append(temp.group())
        sgg_temp = temp_list
            
    sido_list.append(list(set(sido_temp)))
    sgg_list.append(list(set(sgg_temp)))
    emd_list.append(list(set(emd_temp)))

    
news_df['sido'] = sido_list
# news_df['sido'] = news_df['sido'].map(lambda x: list(set(x))).map(lambda x: ','.join(x))
news_df['sgg'] = sgg_list
# news_df['sgg'] = news_df['sgg'].map(lambda x: list(set(x))).map(lambda x: ','.join(x))
news_df['emd'] = emd_list
# news_df['emd'] = news_df['emd'].map(lambda x: list(set(x))).map(lambda x: ','.join(x))

['김천시', '김천시', '김천시']
3
['춘천', '춘천', '춘천', '춘천시', '춘천']
1
['춘천', '춘천시']
1
['목포시', '목포']
1
['양산', '양산시']
1
['홍성군', '홍성군', '홍성군']
3
['기장', '동남', '기장군', '기장군', '기장군', '성동']
3
['완주', '완주군']
1
['목포시', '목포']
1
['중구', '중구', '중구', '중구', '중구']
5
['양주', '양주시']
1
['춘천', '춘천시']
1
['목포시', '목포']
1
['김해', '김해시']
1
['성남시', '영주']
1
['천안시', '천안']
1
['양주시', '양주', '양주', '양주']
1
['괴산군', '괴산']
1
['중구', '중구', '중구', '중구', '중구']
5
['양주', '양주시']
1
['중구', '중구', '중구']
3
['춘천', '춘천시']
1
['구미', '구미', '구미시', '구미']
1
['목포시', '목포']
1
['괴산군', '원주']
1
['양주', '양주시']
1
['김해', '김해시']
1
['성남시', '영주']
1
['양주', '양주', '양주시', '양주']
1
['광산구', '광산구', '광산구']
3
['천안시', '천안']
1
['양주시', '양주', '양주', '양주']
1
['괴산', '괴산군']
1
['문경', '문경시']
1
['횡성', '횡성군', '동해', '가평', '동해']
1
['성남시', '분당', '구로']
1
['태안군', '광명']
1
['중구', '중구', '중구']
3
['예천군', '예천']
1
['청주', '청주시']
1
['정선군', '정선']
1
['청주', '청주시']
1
['영양', '양구', '양양']
1
['충주시', '충주', '충주']
1
['양주시', '연수']
1
['양주시', '연수']
1
['영양', '영주', '영주시']
1
['예천', '영주시']
1
['동구', '달서구', '중구', '중구', '중구']

In [ ]:
news_df

,title,url,text,date,region_keyword,region_keyword2,region_total,sido,sgg,emd
0,"지역난방공사, '광양항 자전거도로 태양광발전소' 준공…그린뉴딜 선도",http://www.cstimes.com/news/articleView.html?i...,"광양 자전거도로 태양광 발전소 한국지역난방공사(사장 황창화, 이하 '한난')는 지...",2021-03-08 08:46,[광양],"[광양, 여수, 광양, 광양]","[광양, 광양, 여수, 광양, 광양]",[],"[여수, 광양]",[]
1,김천시청 주차 건물 준공 고질적 주차문제 해결 희소식,https://news.imaeil.com/Society/20210302164957...,경북 김천시청의 고질적인 주차난이 이달 말이면 시원하게 해결된다. 2019년 5월 ...,2021-03-08 08:46,[김천시],"[경북, 김천시, 김천시]","[김천시, 경북, 김천시, 김천시]",[경북],[김천시],[]
2,"진주시, 물초울공원에 어린이 놀이시설 조성...6월 준공",https://www.nocutnews.co.kr/news/5511560,"도비 10억원, 시비 19억원 등 총 29억원의 예산을 확보해 지난 해 12월 사업...",2021-03-08 08:46,[진주시],[],[진주시],[],[진주시],[]
3,괴산군 첫 수소자동차 충전소 다음달 준공,https://www.inews365.com/news/article.html?no=...,군은 지난달 한국가스기술공사와의 업무 협약에 따라 30억 원(국비 15억 원 포함)...,2021-03-08 08:46,[괴산군],[괴산읍],"[괴산군, 괴산읍]",[],[괴산군],[괴산읍]
4,"서산시, 부석면 행정복지센터 기공식… 2022년 5월 준공",http://www.jbnews.com/news/articleView.html?id...,"신설되는 부석면 행정복지센터는 기존 청사 뒤편에 위치하며, 61억 원을 투입해 8천...",2021-03-08 08:46,"[서산시, 부석면]",[부석면],"[서산시, 부석면, 부석면]",[],[서산시],[부석면]
5,"구례군, 내안교 신축공사 준공 및 농어촌도로 개통",http://www.breaknews.com/790298,전남 구례군이 지역주민의 숙원사업 이였던 지방하천 마산천을 횡단하는 교량을 신축하고...,2021-03-08 08:46,[구례군],"[전남, 구례군, 마산면]","[구례군, 전남, 구례군, 마산면]",[전남],[구례군],[마산면]
6,'정부세종 신청사' 에너지 절약 첨단 기술 집대성… 내년 8월 준공,http://www.ikld.kr/news/articleView.html?idxno...,[국토일보 하종숙 기자] 내년 8월 준공 예정인 '정부세종 신청사'는 에너지 절약 ...,2021-03-08 08:46,[세종],"[세종, 세종]","[세종, 세종, 세종]",[세종],[],[]
7,[춘천]바이오 특화 남춘천산업단지 7월 준공,http://www.kwnews.co.kr/nview.asp?aid=22103070...,바이오특화단지로 조성중인 남춘천산업단지가 오는 7월 준공될 예정이다. 춘천시에 따르...,2021-03-08 08:46,"[춘천, 춘천]","[춘천, 춘천시, 춘천]","[춘천, 춘천, 춘천, 춘천시, 춘천]",[],[춘천시],[]
8,남춘천 일반산업단지 공정률 96% 육박···오는 7월 준공 예정,http://www.ajunews.com/view/20210307120051819,[사진=강원 춘천시 제공] ‘남춘천 일반산업단지’ 준공이 임박했다. 7일 시에 따르...,2021-03-08 08:46,[춘천],"[강원, 춘천시, 춘천, 춘천시, 남산면, 오산, 춘천]","[춘천, 강원, 춘천시, 춘천, 춘천시, 남산면, 오산, 춘천]",[강원],"[춘천, 오산, 춘천시]",[남산면]
9,"완주군, 소양면 농촌중심지 활성화 사업 올해 준공 예정",http://www.jjan.kr/news/articleView.html?idxno...,완주군 소양면 농촌중심지 활성화사업이 올해 준공될 예정이다. 이 사업은 지난 201...,2021-03-08 08:46,"[완주군, 소양면]","[완주군, 소양면, 소양면]","[완주군, 소양면, 완주군, 소양면, 소양면]",[],[완주군],[소양면]


### 준공 시도 시군구 읍면동 전처리
1. 시도, 시군구, 읍면동 리스트 안에 여러 객체가 존재할 때 시도 혹은 시군구 혹은 읍면동으로 끝나는 것을 우선적으로 선별  
    ex) 은평구,영주 => 은평구
2. 공식 명칭으로 변환하기  
    ex) 서울 => 서울특별시
3. [sido, sgg, emd] 리스트와 매칭하여 여러 객체 중 하나의 객체 선택

### 준공 시도 시군구 읍면동 전처리 (2)
1. 글자 길이를 기준으로 선택

In [ ]:
# sgg_list : 두 개의 시 구로 이루어져있는 sgg 리스트
sgg_list = []
for i in open('sgg_two', 'r', encoding = 'utf-8').read().split('\n'):
    sgg_list.append(i.split('\t'))

# 시도, 시군구, 읍면동 값에 많이 들어가 있을 때, 
# 1) sgg에 두 개 이사 들어가 있는 것 중 두 개가 같이 들어가야 하는 것만 남기고 하나의 문자열로 합치기
# 2) 나머지는 가장 긴 단어 한 개만 남기기 ( 단어 길이가 동일한 경우는 남겨두기 )
for i in range(5,8):
    for j in range(0, len(news_df)):
        if len(news_df.iloc[j,i]) > 1:
            temp = []
            count = {}
            
            for word in news_df.iloc[j,i]:
                temp.append(len(word))
            for length in temp:
                try: count[length] += 1
                except: count[length] = 1
                
            if i == 6:
                break_var = False
                for sgg in sgg_list:
                    temp2 = [list(set(news_df.iloc[j,i]).intersection(sgg))]
                    if len(temp2) == 2:
                        news_df.iloc[j,i] = [' '.join(temp2)]
                        break_var = True
                        break
                if max(count.values()) == 1:
                    news_df.iloc[j,i].sort(key=len)
                    news_df.iloc[j,i] = news_df.iloc[j,i][::-1]
                    news_df.iloc[j,i] = [news_df.iloc[j,i][0]]
            else :
                if max(count.values()) == 1:
                    news_df.iloc[j,i].sort(key=len)
                    news_df.iloc[j,i] = news_df.iloc[j,i][::-1]
                    news_df.iloc[j,i] = [news_df.iloc[j,i][0]]

In [ ]:
news_df

,title,url,text,date,region_keyword,region_keyword2,region_total,sido,sgg,emd
0,"지역난방공사, '광양항 자전거도로 태양광발전소' 준공…그린뉴딜 선도",http://www.cstimes.com/news/articleView.html?i...,"광양 자전거도로 태양광 발전소 한국지역난방공사(사장 황창화, 이하 '한난')는 지...",2021-03-08 08:46,[광양],"[광양, 여수, 광양, 광양]","[광양, 광양, 여수, 광양, 광양]",[],"[여수, 광양]",[]
1,김천시청 주차 건물 준공 고질적 주차문제 해결 희소식,https://news.imaeil.com/Society/20210302164957...,경북 김천시청의 고질적인 주차난이 이달 말이면 시원하게 해결된다. 2019년 5월 ...,2021-03-08 08:46,[김천시],"[경북, 김천시, 김천시]","[김천시, 경북, 김천시, 김천시]",[경북],[김천시],[]
2,"진주시, 물초울공원에 어린이 놀이시설 조성...6월 준공",https://www.nocutnews.co.kr/news/5511560,"도비 10억원, 시비 19억원 등 총 29억원의 예산을 확보해 지난 해 12월 사업...",2021-03-08 08:46,[진주시],[],[진주시],[],[진주시],[]
3,괴산군 첫 수소자동차 충전소 다음달 준공,https://www.inews365.com/news/article.html?no=...,군은 지난달 한국가스기술공사와의 업무 협약에 따라 30억 원(국비 15억 원 포함)...,2021-03-08 08:46,[괴산군],[괴산읍],"[괴산군, 괴산읍]",[],[괴산군],[괴산읍]
4,"서산시, 부석면 행정복지센터 기공식… 2022년 5월 준공",http://www.jbnews.com/news/articleView.html?id...,"신설되는 부석면 행정복지센터는 기존 청사 뒤편에 위치하며, 61억 원을 투입해 8천...",2021-03-08 08:46,"[서산시, 부석면]",[부석면],"[서산시, 부석면, 부석면]",[],[서산시],[부석면]
5,"구례군, 내안교 신축공사 준공 및 농어촌도로 개통",http://www.breaknews.com/790298,전남 구례군이 지역주민의 숙원사업 이였던 지방하천 마산천을 횡단하는 교량을 신축하고...,2021-03-08 08:46,[구례군],"[전남, 구례군, 마산면]","[구례군, 전남, 구례군, 마산면]",[전남],[구례군],[마산면]
6,'정부세종 신청사' 에너지 절약 첨단 기술 집대성… 내년 8월 준공,http://www.ikld.kr/news/articleView.html?idxno...,[국토일보 하종숙 기자] 내년 8월 준공 예정인 '정부세종 신청사'는 에너지 절약 ...,2021-03-08 08:46,[세종],"[세종, 세종]","[세종, 세종, 세종]",[세종],[],[]
7,[춘천]바이오 특화 남춘천산업단지 7월 준공,http://www.kwnews.co.kr/nview.asp?aid=22103070...,바이오특화단지로 조성중인 남춘천산업단지가 오는 7월 준공될 예정이다. 춘천시에 따르...,2021-03-08 08:46,"[춘천, 춘천]","[춘천, 춘천시, 춘천]","[춘천, 춘천, 춘천, 춘천시, 춘천]",[],[춘천시],[]
8,남춘천 일반산업단지 공정률 96% 육박···오는 7월 준공 예정,http://www.ajunews.com/view/20210307120051819,[사진=강원 춘천시 제공] ‘남춘천 일반산업단지’ 준공이 임박했다. 7일 시에 따르...,2021-03-08 08:46,[춘천],"[강원, 춘천시, 춘천, 춘천시, 남산면, 오산, 춘천]","[춘천, 강원, 춘천시, 춘천, 춘천시, 남산면, 오산, 춘천]",[강원],"[춘천, 오산, 춘천시]",[남산면]
9,"완주군, 소양면 농촌중심지 활성화 사업 올해 준공 예정",http://www.jjan.kr/news/articleView.html?idxno...,완주군 소양면 농촌중심지 활성화사업이 올해 준공될 예정이다. 이 사업은 지난 201...,2021-03-08 08:46,"[완주군, 소양면]","[완주군, 소양면, 소양면]","[완주군, 소양면, 완주군, 소양면, 소양면]",[],[완주군],[소양면]


In [ ]:
# 날짜 뽑기
dt_list = []

idx = 0
for t in news_df['title']: 
    dt_list.append(' '.join(re.findall(r"(올해|내년|내달|\d{1,2}월|\d{4}년|\d{1,2}일|상반기|하반기)", t)))

news_df['dt_test'] = dt_list

news_df['dt_test'] = news_df['dt_test'].map(lambda x: re.sub('(^\d{1,2}월)', date.split('-')[0] + '년 ' + r'\1',
                                                               re.sub('올해',date.split('-')[0] + '년',
                                                                       re.sub('내년',str(int(date.split('-')[0]) + 1) + '년',
                                                                              re.sub('내달',str(int(date.split('-')[1]) + 1) + '월',x)))))

In [ ]:
news_df[['title','text', 'sido', 'sgg', 'emd','dt_test']]

,title,text,sido,sgg,emd,dt_test
0,"지역난방공사, '광양항 자전거도로 태양광발전소' 준공…그린뉴딜 선도","광양 자전거도로 태양광 발전소 한국지역난방공사(사장 황창화, 이하 '한난')는 지...",[],"[여수, 광양]",[],
1,김천시청 주차 건물 준공 고질적 주차문제 해결 희소식,경북 김천시청의 고질적인 주차난이 이달 말이면 시원하게 해결된다. 2019년 5월 ...,[경북],[김천시],[],
2,"진주시, 물초울공원에 어린이 놀이시설 조성...6월 준공","도비 10억원, 시비 19억원 등 총 29억원의 예산을 확보해 지난 해 12월 사업...",[],[진주시],[],2021년 6월
3,괴산군 첫 수소자동차 충전소 다음달 준공,군은 지난달 한국가스기술공사와의 업무 협약에 따라 30억 원(국비 15억 원 포함)...,[],[괴산군],[괴산읍],
4,"서산시, 부석면 행정복지센터 기공식… 2022년 5월 준공","신설되는 부석면 행정복지센터는 기존 청사 뒤편에 위치하며, 61억 원을 투입해 8천...",[],[서산시],[부석면],2022년 5월
5,"구례군, 내안교 신축공사 준공 및 농어촌도로 개통",전남 구례군이 지역주민의 숙원사업 이였던 지방하천 마산천을 횡단하는 교량을 신축하고...,[전남],[구례군],[마산면],
6,'정부세종 신청사' 에너지 절약 첨단 기술 집대성… 내년 8월 준공,[국토일보 하종숙 기자] 내년 8월 준공 예정인 '정부세종 신청사'는 에너지 절약 ...,[세종],[],[],2022년 8월
7,[춘천]바이오 특화 남춘천산업단지 7월 준공,바이오특화단지로 조성중인 남춘천산업단지가 오는 7월 준공될 예정이다. 춘천시에 따르...,[],[춘천시],[],2021년 7월
8,남춘천 일반산업단지 공정률 96% 육박···오는 7월 준공 예정,[사진=강원 춘천시 제공] ‘남춘천 일반산업단지’ 준공이 임박했다. 7일 시에 따르...,[강원],"[춘천, 오산, 춘천시]",[남산면],2021년 7월
9,"완주군, 소양면 농촌중심지 활성화 사업 올해 준공 예정",완주군 소양면 농촌중심지 활성화사업이 올해 준공될 예정이다. 이 사업은 지난 201...,[],[완주군],[소양면],2021년


In [ ]:
news_df[news_df['region_keyword'].map(lambda x: len(x)) == 0]

,title,url,text,date,region_keyword,region_keyword2,region_total,sido,sgg,emd,dt_test
12,"[TV북마크] ‘구해줘!홈즈’ 아이돌 출신 예비 부부 출연, 슬리피·딘딘 쪽 선택 ...",https://sports.donga.com/article/all/20210308/...,1980년에 준공된 구옥이지만 2018년 부분 리모델링을 마친 상태였다. 원목 웨인...,2021-03-08 08:46,[],[],[],[],[],[],
17,‘구해줘!홈즈’ 前 아이돌 커플 신혼집 찾는다,http://sports.khan.co.kr/news/sk_index.html?ar...,1980년에 준공된 구옥이지만 2018년 부분 리모델링을 마친 매물로 실내 공간을 ...,2021-03-08 08:46,[],[],[],[],[],[],
22,"[단독] 車 카메라 모듈 1위 엠씨넥스, '코스닥→코스피' 이전 상장한다",https://www.hankyung.com/economy/article/20210...,추가 투자를 위해 운영자금 확보가 필요한 상황이다. 회사 관계자는 “2019년 말 ...,2021-03-08 08:46,[],[],[],[],[],[],
31,"영남대, 삼익THK㈜ 진영환 회장 명예박사 학위 수여",https://news.imaeil.com/Education/202103041209...,영남대는 진 회장의 발전기금으로 2019년 영남대 기계관 1층 로비에 '삼익THK ...,2021-03-08 08:46,[],[대구],[대구],[대구],[],[],
43,영남알프스 케이블카 우선협상대상자에 세진중공업,http://www.ksilbo.co.kr/news/articleView.html?...,"군은 조만간 세진중공업과 협상을 시작해 연내 협약을 체결하고, 환경영향평가 등 사전...",2021-03-08 08:46,[],[],[],[],[],[],
49,열악한 시·군 재정 확충에 적극 행정 촉구,http://www.hidomin.com/news/articleView.html?i...,"영주 첨단베어링 국가산업단지는 문재인 대통령 국정과제 지역 공약에 포함, 2018년...",2021-03-08 08:46,[],[영주],[영주],[],[영주],[],
51,"[단독] ""탈원전은 곧 전기료 상승"" 반발 귀막은 정부",http://news.mk.co.kr/newsRead.php?no=218263&ye...,실제 한수원이 작성한 답변서에는 '정부 정책을 고려할 때 준공 일정에 불확실성이 있...,2021-03-08 08:46,[],"[수원, 수원]","[수원, 수원]",[],[수원],[],
54,"내진설계, Warranty 10년이 절실하다",http://www.newsmaker.or.kr/news/articleView.ht...,준공되고 있는 지금의 진행 또한 코로나시대에 발맞추어 거대한 스케일로 확대되고 있다...,2021-03-08 08:46,[],[],[],[],[],[],
57,마을과 축산이 만나 기후위기를 준비했다,http://www.ikpnews.net/news/articleView.html?i...,될 '원천에너지전환센터'가 준공됐다. 마을은 이곳에서 나오는 폐열로 난방비를 절감한...,2021-03-08 08:46,[],[],[],[],[],[],
58,'의사면허 취소법' 막기 위한 의협회장 후보들 대책은?,http://www.monews.co.kr/news/articleView.html?...,김동석 후보는 의협회관 준공과 관련해 합법적인 기금을 받는 방안을 강구해야 한다고 ...,2021-03-08 08:46,[],[이동],[이동],[],[],[이동],


In [ ]:
# news_df.to_excel('test.xlsx')